<a href="https://colab.research.google.com/github/bearbearyu1223/llm-fine-tuning-playground/blob/main/finetune_falcon_7b_conversation_summarization_12_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune [Falcon-7b-sharded model](https://huggingface.co/vilsonrodrigues/falcon-7b-sharded)** on [samsum](https://huggingface.co/datasets/samsum) can be found on Hugging Face.
Links to both the model and dataset are in the notebook.


## Set Up the Development Enviroment

In [1]:
!pip install huggingface_hub==0.19.4
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install transformers==4.36.0
!pip install datasets==2.15.0 Tokenizers==0.15.0
!pip install -q bitsandbytes wandb
!pip install py7zr

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#all imports
import torch
import time
import numpy as np
from huggingface_hub import notebook_login
from datasets import load_dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training, TaskType
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


##Notebook connection to Hugging face

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
notebook_login()

##Loading the Base Model and Setting Up bitsandbytes config

We will use sharded version of falcon-7b model.


In [5]:
model_name = "vilsonrodrigues/falcon-7b-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

##Loading the tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
print("pad token id : {}".format(tokenizer.pad_token_id))
print("eos token id : {}".format(tokenizer.eos_token_id))

pad token id : None
eos token id : 11


In [12]:
if tokenizer.pad_token_id is None:
  tokenizer.pad_token = tokenizer.eos_token
  model.config.pad_token_id = model.config.eos_token_id

##Loading the dataset from hugging face and Formatting the Training Dataset

---



In [13]:
dataset_name = "samsum"
dataset = load_dataset(dataset_name)

train_dataset = dataset['train']
eval_dataset = dataset['validation']
test_dataset = dataset['test']
dataset

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [14]:
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 90 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 90))
print(f"Max source length: {max_source_length}")

tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max source length: 315


Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max target length: 46


##Setting up the LoRA config

In [15]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 16 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [23]:
peft_model.print_trainable_parameters()
#trainable params: 32,636,928 || all params: 6,954,357,632 || trainable%: 0.4693018352956629

trainable params: 32,636,928 || all params: 6,954,357,632 || trainable%: 0.4693018352956629


##Load the trainer

In [16]:
output_dir = "falcon_7b_LoRA_r16_dialogue_summarization_12_17_2023"
per_device_train_batch_size = 32 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 20
logging_steps = 20
learning_rate = 2e-4
max_grad_norm = 0.1
warmup_ratio = 0.01
lr_scheduler_type = "cosine" #"constant"
num_train_epochs = 3
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    num_train_epochs=num_train_epochs,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to="wandb"
)

##Passing arguments to the SFTT trainer

In [17]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['id'])):
        instruction = "Summarize this Dialogue."
        input = example['dialogue'][i]
        output = example['summary'][i]
        prompt="### Instruction:\n{instruction}\n\n### Dialogue:\n{input}\n\n### Summary:\n{output}".format(instruction=instruction, input=input, output=output)
        output_texts.append(prompt)
    return output_texts

In [18]:
max_seq_length = 512

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    formatting_func=formatting_prompts_func,
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    args=training_arguments,
)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [19]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

##Train the model

You can check your training time if you are doing multiple experiments

In [20]:
import os
import time
import wandb
os.environ["WANDB_SILENT"] = "true"
wandb.init(project="falcon-7b-peft-lora-r16-dialogue-summarization")

<IPython.core.display.Javascript object>

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


In [24]:
# silence the warnings. Please re-enable for inferenc
peft_model.config.use_cache = False

In [25]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
20,1.865100
40,1.736300
60,1.666100
80,1.661300
100,1.717300
120,1.625800
140,1.603000
160,1.683700
180,1.603700
200,1.556900


TrainOutput(global_step=345, training_loss=1.6339570736539537, metrics={'train_runtime': 3812.0841, 'train_samples_per_second': 11.594, 'train_steps_per_second': 0.091, 'total_flos': 3.6187889779597824e+17, 'train_loss': 1.6339570736539537, 'epoch': 2.99})

##Push to hub

In [26]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/131M [00:00<?, ?B/s]

'https://huggingface.co/bearbearyu1223/falcon_7b_LoRA_r16_dialogue_summarization_12_17_2023/tree/main/'

##Inference

In [27]:
# Loading PEFT model
PEFT_MODEL = "bearbearyu1223/"+ output_dir
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
if peft_tokenizer.pad_token_id is None:
   peft_tokenizer.pad_token = peft_tokenizer.eos_token
   peft_model.config.pad_token_id = peft_model.config.eos_token_id

adapter_config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/131M [00:00<?, ?B/s]

In [28]:
# Generate responses from both orignal model and fine-tuned model
def get_response(dialogue, max_new_tokens=10, verbose=False):
  prompt= "### Instruction:\n{instruction}\n\n### Dialogue:\n{dialogue}\n\n### Summary:\n".format(instruction="Summarize the Dialogue below.", dialogue=dialogue)
  if verbose:
    print(prompt)
  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to(torch.device("cuda:0"))
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=max_new_tokens, pad_token_id = peft_tokenizer.eos_token_id,
                                                                                                         eos_token_id = peft_tokenizer.eos_token_id,
                                                                                                         attention_mask = peft_encoding.attention_mask,
                                                                                                         temperature=0.1, top_p=0.9,  repetition_penalty=10.0, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)
  return peft_text_output

In [29]:
import re

def remove_incomplete_sentences(text):

  # Compile the regular expression for complete sentences.
  complete_sentence_regex = re.compile(r"(^.*[\.\?!]|^\S[^.\?!]*)")

  # Find all of the complete sentences in the text.
  complete_sentences = complete_sentence_regex.findall(text)
  text = " ".join(complete_sentences)

  # Return the text with the incomplete sentences removed.
  return text


text = "This is a sentence! This is not"
remove_incomplete_sentences(text)

'This is a sentence!'

In [33]:
!pip install --user -U nltk

In [39]:
import transformers
import nltk
nltk.download('punkt')

test_index=70
dialogue=test_dataset[test_index]['dialogue']
summary=test_dataset[test_index]['summary']
condense_rate=0.5

max_new_tokens=max(int(len(nltk.word_tokenize(dialogue))*condense_rate), len(nltk.word_tokenize(summary)))
peft_output=get_response(dialogue, max_new_tokens)
sub = "### Summary:"
peft_summary = peft_output.split(sub)[1]
post_processed_peft_model_summary = remove_incomplete_sentences(peft_summary.strip())

pipeline = transformers.pipeline(
    "text-generation",
    model=peft_base_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
    formatting_prompts_func(test_dataset)[test_index],
    max_new_tokens=max_new_tokens,
    temperature=0.1, top_p=0.9,  repetition_penalty=10.0, num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
post_processed_base_model_summary= remove_incomplete_sentences(peft_summary.strip())
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print('BASELINE HUMAN SUMMARY:')
print(summary)
print(dash_line)
print('BASE MODEL SUMMARY:')
print(sequences[0]['generated_text'])
print(dash_line)
print('PEFT MODEL SUMMARY')
print(post_processed_peft_model_summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Ali left his wallet at Mohammad's place. Mohammad'll bring it to uni tomorrow.
---------------------------------------------------------------------------------------------------
BASE MODEL SUMMARY:
### Instruction:
Summarize this Dialogue.

### Dialogue:
Ali: I think I left my wallet at your place yesterday. Could you check? 
Mohammad: Give me a sec, I'll have a look around my room.
Ali: OK.
Mohammad: Found it!
Ali: Phew, I don't know what I'd do if it wasn't there. Can you bring it to uni tomorrow?
Mohammad: Sure thing.

### Summary:
Ali left his wallet at Mohammad's place. Mohammad'll bring it to uni tomorrow.  Ali is relieved.  Mohammad found the wallet in his room.  Mohammad will return it to Ali tomorrow.   Ali doesn't want to lose his wallet again. 
---------------------------------------------------------------------------------------------------
PEFT MODE

### Evaluation

In [31]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install evaluate
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=e11367e67c145d9a2af72eab7e90b9990726a4738d9636ec76366f077c868225
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [41]:
from tqdm import tqdm

references = []
predictions = []

for sample in tqdm(test_dataset):
  summary=sample['summary']
  dialogue=sample['dialogue']
  max_new_tokens=max(int(len(nltk.word_tokenize(dialogue))*condense_rate), len(nltk.word_tokenize(summary)))
  peft_output=get_response(dialogue, max_new_tokens)
  sub = "### Summary:"
  peft_summary = peft_output.split(sub)[1]
  post_processed_peft_model_summary = remove_incomplete_sentences(peft_summary.strip())
  references.append(summary)
  predictions.append(post_processed_peft_model_summary)

100%|██████████| 819/819 [1:14:52<00:00,  5.49s/it]


In [42]:
import evaluate

metric = evaluate.load('rouge')
results = metric.compute(predictions=predictions, references=references)

print(f"Rogue1: {results['rouge1']* 100:2f}%")
print(f"rouge2: {results['rouge2']* 100:2f}%")
print(f"rougeL: {results['rougeL']* 100:2f}%")
print(f"rougeLsum: {results['rougeLsum']* 100:2f}%")

Rogue1: 33.948824%
rouge2: 10.162770%
rougeL: 24.564262%
rougeLsum: 24.577140%
